In [27]:
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
import geodatasets
import matplotlib.pyplot as plt
import plotly.express as px


In [ ]:
crash_df = pd.read_csv('../Resources/Montgomery_Crash_Data.csv', index_col='Crash Date/Time', parse_dates=True)
crash_df.head()

In [ ]:
crash_df.info()

In [ ]:
montgomery_gdf = gpd.GeoDataFrame(crash_df, geometry=gpd.points_from_xy(crash_df.Longitude, crash_df.Latitude), crs='EPSG:4326').sort_index()
montgomery_gdf.loc['2020']


In [ ]:
county_gdf = gpd.read_file('../Resources/Montgomery_County.json')
county_gdf.head()

In [ ]:
base = county_gdf.plot(color='white', edgecolor='black')
montgomery_gdf.loc['2021-06'].plot(ax=base, marker='o', color='red', markersize=2)

In [ ]:
filtered_data.loc[conditional]['Surface Condition'].value_counts()

In [ ]:
filtered_data.loc['2020']['Injury Severity'].value_counts()

In [ ]:
m = folium.Map(location=[39.1364, -77.1715], tiles="Cartodb dark_matter", zoom_start=11) 

for _, r in county_gdf.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r["geometry"]).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j, style_function=lambda x: {"fillColor": "none"})
    folium.Popup(r["County"]).add_to(geo_j)
    geo_j.add_to(m)

column = 'Surface Condition'
value = 'ICE'
year = '2015'

column = 'Weather'
value = 'RAIN'
year = '2015'


column = 'Injury Severity'
value = 'FATAL INJURY'
year = '2020'

filtered_data = montgomery_gdf.loc[year:'2024']
conditional = filtered_data[column] == value
heat_data = [[point.xy[1][0], point.xy[0][0]] for point in filtered_data.loc[conditional].geometry]
plugins.HeatMap(heat_data, min_opacity=.2, radius=20,zoom_start=2).add_to(m)


for point in heat_data:
    folium.CircleMarker(point, radius=.2, fill_color="black", color="blue").add_to(m)

# fg = folium.FeatureGroup(name="Icon collection", show=False).add_to(m)    
folium.LayerControl().add_to(m)

m

In [ ]:
fig = px.density_mapbox(montgomery_gdf.loc['2020'], lat='Latitude', lon='Longitude', z='Vehicle Year', radius=15
                        ,
                        center = {'lat':39.1364,'lon':-77.1715}, zoom= 11, mapbox_style='carto-darkmatter', height=900, opacity=.5 )
fig.show()

In [ ]:
weeks = crash_df.index.isocalendar().week.sort_values().unique()

plt.bar(weeks, crash_df.index.isocalendar().week.value_counts().sort_index())
plt.title('Weekly Crashes')
plt.xlabel('Weeks')
plt.ylabel('Counts')
plt.show()


weekly_crash_count = crash_df.groupby([crash_df.index.isocalendar().year, crash_df.index.isocalendar().week])['Report Number'].count().reset_index()
plt.bar(weeks, weekly_crash_count.groupby('week')['Report Number'].mean().round())
plt.title('Weekly Crashes')
plt.xlabel('Week')
plt.ylabel('Average Crashes')
plt.show()

weekly_crash_count


In [ ]:
days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']  # crash_df.index.isocalendar().day.sort_values().unique()

# plt.figure(figsize=(10,8))
plt.bar(days, crash_df.index.isocalendar().day.value_counts().sort_index())
plt.title('Daily Crashes')
plt.xlabel('Day')
plt.ylabel('Counts')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.show()


daily_crash_count = crash_df.groupby([crash_df.index.isocalendar().year, crash_df.index.isocalendar().week, crash_df.index.isocalendar().day])['Report Number'].count().reset_index()

# plt.figure(figsize=(10,8))
plt.bar(days, daily_crash_count.groupby('day')['Report Number'].mean().round())
plt.title('Daily Average Crashes')
plt.xlabel('Day')
plt.ylabel('Average Crashes')
plt.xticks(rotation=45, ha='right', rotation_mode='anchor')
plt.show()

In [ ]:
years = crash_df.index.isocalendar().year.sort_values().unique()
plt.bar(years, montgomery_gdf.index.isocalendar().year.value_counts().sort_index())
plt.grid(linestyle = '-' )
plt.title('Yearly Crashes')
plt.xlabel('Year')
plt.ylabel('Counts')
plt.show()